In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import cudf
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.rfr.rfr_cv_trainer as cv
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [2]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = cudf.read_parquet("../artifacts/features/base/tr_df1.parquet").astype("float32")
test_df1 = cudf.read_parquet("../artifacts/features/base/test_df1.parquet").astype("float32")

In [ ]:
# Check tuning results
study_name = "rfr_v1"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()
study.summarize_studies()

In [ ]:
# Create OOF and test predictions
importlib.reload(cv)
params = {
    "n_estimators": 70,
    "max_depth": 15
}
trainer = cv.RFRCVTrainer(params=params)
oof, test_preds = trainer.fit(tr_df1, test_df1)
np.save("../artifacts/preds/base/oof_single_1.npy", oof)
np.save("../artifacts/preds/base/test_single_1.npy", test_preds)

In [ ]:
te.send_telegram_message("RFR Training complete!")